# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to StepFun API
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [StepFun API Key](https://platform.stepfun.com/interface-key)

In [4]:
%pip install python-dotenv
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
api_key = os.environ['STEP_API_KEY']
base_url = os.environ['STEP_BASE_URL']
llm_model = os.environ['STEP_LLM_MODEL']
print(api_key)
print(base_url)
print(llm_model)

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in this nootbook.

## Chat API : StepFun

Let's start with a direct API calls to StepFun.

In [6]:
client = OpenAI(api_key=api_key, base_url=base_url)
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    return response.choices[0].message.content


In [7]:
get_completion("What is 1+1?")

'1 + 1 = 2.\n\nThis is one of the basic arithmetic operations, known as addition. It involves combining two numbers to find a total. In this case, we are adding the numbers 1 and 1 together, which results in the sum of 2. This can be represented using the plus sign (+) and the equals sign (=), as shown above.'

In [8]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [9]:
style = """American English \
in a calm and respectful tone
"""

In [10]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [11]:
response = get_completion(prompt)

In [12]:
print(response)

Certainly! Here's the text translated into American English in a calm and respectful tone:

```
Wow, I'm really frustrated that the lid of my blender flew off and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend.
```


## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
#!pip install --upgrade langchain

### Model

In [13]:
from langchain.chat_models import ChatOpenAI

In [68]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
# llm_model="step-1v-8k"
chat = ChatOpenAI(temperature=0.0, model=llm_model, api_key=api_key, base_url=base_url)
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1159a6050>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1159fee50>, model_name='step-1v-8k', temperature=0.0, openai_api_key='6YQ9D8bhClOmzSL80CUwOmR10f4SzLJxwVhiHTroW1m0X6lZEbADHORZvy1nQonev', openai_api_base='https://api.stepfun.com/v1', openai_proxy='')

### Prompt template

In [15]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [16]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [17]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [18]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [19]:
customer_style = """American English \
in a calm and respectful tone
"""

In [20]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [21]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [22]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [23]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [24]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [25]:
print(customer_response.content)

```
Alright, I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, buddy.
```


In [26]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [27]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [28]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [29]:
service_response = chat(service_messages)
print(service_response.content)

```
Ahoy there, me hearty! 'Tis a sorry tale, but the warranty be not for cleanin' up yer kitchen. Ye see, 'twas yer own fault fer misusin' the blender, fergettin' the lid afore ye set it a-spinnin'. 'Tis a hard knock, but that be the way of it. Fare thee well, and may the winds be ever at yer back!
```


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [30]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [31]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [32]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [33]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=llm_model, api_key=api_key, base_url=base_url)
response = chat(messages)
print(response.content)

To extract the information from the given text, we can use natural language processing techniques to identify the relevant sentences and keywords. Here's the output in JSON format:

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
  ]
}
```

Here's how we can extract the information:

1. **gift**: We can look for keywords like "gift", "present", or "for someone else" to determine if the item was purchased as a gift. In this case, the text mentions "my wife's anniversary present", so we can conclude that the item was purchased as a gift.

2. **delivery_days**: We can look for keywords like "arrived", "days", or "delivery" to determine how many days it took for the product to arrive. In this case, the text mentions "It arrived in two days", so we can conclude that it took 2 days for the product to arrive.

3. **price_value**: We can look for keywor

In [34]:
type(response.content)

str

In [35]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [36]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [74]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days did it take for the product to arrive? If this information is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any sentences about the value or price, and output them as a comma separated Python List.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [75]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [76]:
format_instructions = output_parser.get_format_instructions()

In [77]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python List.
}
```


In [78]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python List.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [79]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python List.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [80]:
response = chat(messages)

In [81]:

print(response.content)
print(type(response.content))


```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```
<class 'str'>


<!-- step-1-256k response -->
```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "['It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.']"
}
```

In the given text, the item was purchased as a gift for someone else (the wife), it took 2 days for the product to arrive, and there is one sentence about the value or price of the item.


<!-- step-1v-8k response -->
```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


<!-- step-1v-32k response -->
```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [82]:
output_dict = output_parser.parse(response.content)

In [83]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [84]:
type(output_dict)

dict

In [85]:
output_dict.get('delivery_days')

'2'

Reminder: Download your notebook to you local computer to save your work.